In [14]:
# Pytorch basic learning
from __future__ import print_function
import torch

print(torch.tensor([5,3]))
print(torch.rand(5,3))
x = torch.tensor([5,3])
print(x.new_ones(5, 3, dtype=torch.double))
print(torch.randn_like(x, dtype=torch.float))

x = torch.tensor([0.8754]) # 指定一个一维向量
if torch.cuda.is_available():
    device = torch.device("cuda")          # 指定GPU设备
    y = torch.ones_like(x, device=device)   # 直接在GPU上创建一个张量
    x = x.to(device)                     # 将x从CPU移动到GPU
    z = x + y
    print(z)
    print(z.to("cpu", torch.double))       # 转移数据到CPU并指定数据类型
    
from torch.autograd import Variable
x = Variable(torch.Tensor([1]), requires_grad=True) # requires_grad，用于决定是否参与误差反向传播或梯度计算

x = torch.rand(3,2)	# 随机生成一个3行2列的矩阵
y = torch.rand(3,1)    # 随机生成一个3行1列的矩阵
print(x)
print(x.shape, y.shape, (x+y).shape)  # x+y操作使用了广播机制
x[:,0] = 1 # 切片操作
print(x)
print(x.view(2,3)) # 以2行3列的形式显示数据
z = torch.randn(1)    # 生成1个随机数
print(z, z.item(), type(z.item()))

a = torch.ones(5)
print(a, type(a))
b = a.numpy()
print(b, type(b))
a.add_(1)   # 逐元素操作，且结果直接作用于操作数本身
print(a, b) # b随a的结果变化而变化

tensor([5, 3])
tensor([[0.0767, 0.3355, 0.2417],
        [0.6150, 0.4488, 0.5436],
        [0.1684, 0.8295, 0.2339],
        [0.9318, 0.1158, 0.3878],
        [0.3507, 0.4833, 0.7704]])
tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]], dtype=torch.float64)
tensor([0.8199, 0.7962])
tensor([[0.4910, 0.0170],
        [0.6398, 0.7626],
        [0.6315, 0.1557]])
torch.Size([3, 2]) torch.Size([3, 1]) torch.Size([3, 2])
tensor([[1.0000, 0.0170],
        [1.0000, 0.7626],
        [1.0000, 0.1557]])
tensor([[1.0000, 0.0170, 1.0000],
        [0.7626, 1.0000, 0.1557]])
tensor([-0.4218]) -0.4218190908432007 <class 'float'>
tensor([1., 1., 1., 1., 1.]) <class 'torch.Tensor'>
[1. 1. 1. 1. 1.] <class 'numpy.ndarray'>
tensor([2., 2., 2., 2., 2.]) [2. 2. 2. 2. 2.]


In [16]:
# PyTorch——回归模型
import torch
from torch.autograd import Variable
# 训练数据
x_data = Variable(torch.Tensor([[1.0], [2.0], [3.0]]))
y_data = Variable(torch.Tensor([[2.0], [4.0], [6.0]]))
class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.linear = torch.nn.Linear(1, 1)
        self.sigmoid = torch.nn.Sigmoid()
    def forward(self, x):
        y_pred = self.sigmoid(self.linear(x))
        return y_pred
# model
model = Model()
criterion = torch.nn.BCELoss(size_average=False)  # 损失函数
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
# 训练：前向反馈，损失，后向传播，step
# training loop
for epoch in range(500):
    # forward pass
    y_pred = model(x_data)
    # compute loss
    loss = criterion(y_pred, y_data)
    if epoch % 20 == 0:
        print(epoch, loss.data)
    # zero gradients
    optimizer.zero_grad()
    #perform backward pass
    loss.backward()
    # update weights
    optimizer.step()
# after training
hour_var = Variable(torch.Tensor([[0.5]]))
print("predict (after training)", 0.5, model.forward(hour_var).data[0][0])
hour_var = Variable(torch.Tensor([[7.0]]))
print("predict (after training)", 7.0, model.forward(hour_var).data[0][0])

0 tensor(10.4088)
20 tensor(-109.0229)
40 tensor(-193.2424)
60 tensor(-231.5057)
80 tensor(-231.9091)
100 tensor(-232.3084)
120 tensor(-232.7139)
140 tensor(-233.0987)
160 tensor(-233.4940)
180 tensor(-233.8995)
200 tensor(-234.2825)
220 tensor(-234.6880)
240 tensor(-235.0446)
260 tensor(-235.6043)
280 tensor(-235.8920)
300 tensor(-236.2974)
320 tensor(-236.9906)
340 tensor(-236.9906)
360 tensor(-236.9906)
380 tensor(-248.6792)
400 tensor(-248.6792)
420 tensor(-248.6792)
440 tensor(-248.6792)
460 tensor(-248.6792)
480 tensor(-248.6792)
predict (after training) 0.5 tensor(1.0000)
predict (after training) 7.0 tensor(1.)


In [19]:
# PyTorch——世界人口的线性回归
import pandas as pd
import torch
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Agg')

current_path = r'C:/Users/Lenovo/Desktop/data/data'
listName = current_path + r'/World_population_estimates.txt'
df = pd.read_csv(listName, header=None, sep='\t')
years = torch.tensor(df.iloc[:, 0], dtype=torch.float32)
df[1] = df.apply(lambda x: float(''.join(x[1].split(',')[:])), axis=1)
populations = torch.tensor(df[1], dtype=torch.float32)

x = torch.stack([years, torch.ones_like(years)], 1)
y = populations
# 最小二乘法线性回归
wr, _ = torch.gels(y, x)
slope, intercept = wr[:2, 0]
result = 'population = {:.2e}*year + {:.2e}'.format(slope, intercept)
print('result: ' + result)

# 正规方程法做线性回归
w = x.t().mm(x).inverse().mm(x.t()).mv(y)
slope, intercept = w
result = 'population = {:.2e}*year+{:.2e}'.format(slope, intercept)
print('result:' + result)

#绘制线性回归的结果
plt.xlabel('Year')
plt.ylabel('Population')
# 绘制散点
plt.scatter(years,populations,s=0.1,label='actual',color='k')
# 预测
estimates = [slope*year + intercept for year in years]
#绘制直线
plt.plot(years.numpy(),estimates,label=result,color='k')
plt.legend()
plt.savefig(current_path+'result.png')

result: population = 7.53e+07*year + -1.45e+11
result:population = 7.55e+07*year+-1.45e+11


ImportError: DLL load failed: 找不到指定的模块。